In [1]:
import pandas as pd
import pdfplumber
import re
import torch

In [2]:
# Loading data from pdf
pdf_texts = []

pdf_files = ["./Data/LLM/danske-boligprisindeks-1938-2017-marc-lund-andersen-bvc.pdf", 
             "./Data/LLM/Fremtidens-boligmarked-i-2030-35-Rapport.pdf", 
             "./Data/LLM/Ejendomssalg_2023.pdf",
             "./Data/LLM/Hvem er det der køber enfamiliehuse.pdf",
             "./Data/LLM/Hver anden boligkøber er førstegangskøber.pdf",
             "./Data/LLM/Sæson på boligmarkedet.pdf",
             "./Data/LLM/1403631_Kandidatafhandling_kontraktnr_23403.pdf"]

for pdf_file in pdf_files:
    with pdfplumber.open(pdf_file) as pdf:
        pdf_text = ""
        for page in pdf.pages:
            pdf_text += page.extract_text()
        pdf_texts.append(pdf_text)

In [3]:
print('%.1000s' % pdf_texts[0])

BOLIGØKONOMISK
VIDENCENTER
WORKING PAPER — JANUAR 2019
Danske boligpris-
indeks 1938-2017
samt historiske data om boligmarkedetDanske boligprisindeks 1938-2017
samt historiske data om boligmarkedet
Marc Lund Andersen
Seniorøkonom
Boligøkonomisk Videncenter
E-mail: man@bvc.dk
Januar 2019
Boligøkonomisk VidencenterForord
Arbejdspapiret omhandler konstruktionen af en række nye boligprisindeks. Indeksene sammenkæder
eksisterende indeks med nye indeks, som er konstrueret til formålet af Boligøkonomisk Videncenter.
Boligøkonomisk Videncenter offentliggør I forvejen boligprisindeks med start i 1992. Indeksene ajourføres
en gang i kvartalet og præsenteres løbende i publikationen Bolig&Tal, som startede tilbage i 2014. Det er
samme metode, som er benyttet til at konstruere de nye unikke prisindeks, som præsenteres her. Metoden
kendes under navnet repeat-sales metoden og er karakteriseret ved, at prisindeks dannes ved brug af
såkaldte salgspar, hvor der kan observeres data for minimum to handler

In [4]:
# Cleaning the pdf text
def clean_pdf_text(text):
    text = text.replace('\n', ' ') # Replaces all \n with a whitespace
    text = re.sub(r'\.{2,}', ' ', text) # Replaces 2 or more consecutive periods with a whitespace
    text = re.sub(r'\s+', ' ', text) # Removes multpiple whitespaces in a row
    return text
    
texts = [clean_pdf_text(s) for s in pdf_texts]

In [5]:
print('%.1000s' % texts[0])

BOLIGØKONOMISK VIDENCENTER WORKING PAPER — JANUAR 2019 Danske boligpris- indeks 1938-2017 samt historiske data om boligmarkedetDanske boligprisindeks 1938-2017 samt historiske data om boligmarkedet Marc Lund Andersen Seniorøkonom Boligøkonomisk Videncenter E-mail: man@bvc.dk Januar 2019 Boligøkonomisk VidencenterForord Arbejdspapiret omhandler konstruktionen af en række nye boligprisindeks. Indeksene sammenkæder eksisterende indeks med nye indeks, som er konstrueret til formålet af Boligøkonomisk Videncenter. Boligøkonomisk Videncenter offentliggør I forvejen boligprisindeks med start i 1992. Indeksene ajourføres en gang i kvartalet og præsenteres løbende i publikationen Bolig&Tal, som startede tilbage i 2014. Det er samme metode, som er benyttet til at konstruere de nye unikke prisindeks, som præsenteres her. Metoden kendes under navnet repeat-sales metoden og er karakteriseret ved, at prisindeks dannes ved brug af såkaldte salgspar, hvor der kan observeres data for minimum to handler

# Using Vector Database

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [7]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\Christoffer\.pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\Christoffer\.pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
persist_directory = './Data/chroma/'

vectordb = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

vectordb.persist()

vectordb._collection.count()

C:\Users\Christoffer\.pyenv\pyenv-win\versions\3.12.3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


7

In [10]:
question1 = 'Stiger eller falder huspriser i Danmark?'
answer1 = vectordb.max_marginal_relevance_search(question1, k=2, fetch_k=4)
for d in answer1:
    print('%.1000s' % d.page_content)

Det danske boligmarked - Er der grund til bekymring? Kandidatafhandling Finansiering og Regnskab Oskar Falch Rønager (110797) 16. maj 2022 Stefan Agergaard Petersen (108650) 266.317 anslag / 119 sider Vejleder: Karsten BeltoftAbstract This thesis strives to examine the current situation on the Danish housing market. Hereby, if there are indicators that raise concern for a housing bubble. Furthermore, different initiatives which can minimize this concern will be addressed. This topic has been examined as housing prices have increased rapidly which can result in concern. The research of this paper is based on quantitative data from public resources and qualitative interviews. The qualitative data has been used as inspiration for topics to investigate and as argumentation for choices throughout the thesis. The acknowledged housing bubble from the mid- ’00s has been examined to benchmark with the current situation. Based on the latest price increase in all regions, it is assessed that the 

In [11]:
question2 = 'Hvilke steder stiger danske huspriser mest?'
answer2 = vectordb.max_marginal_relevance_search(question2, k=2, fetch_k=4)
for d in answer2:
    print('%.1000s' % d.page_content)

Det danske boligmarked - Er der grund til bekymring? Kandidatafhandling Finansiering og Regnskab Oskar Falch Rønager (110797) 16. maj 2022 Stefan Agergaard Petersen (108650) 266.317 anslag / 119 sider Vejleder: Karsten BeltoftAbstract This thesis strives to examine the current situation on the Danish housing market. Hereby, if there are indicators that raise concern for a housing bubble. Furthermore, different initiatives which can minimize this concern will be addressed. This topic has been examined as housing prices have increased rapidly which can result in concern. The research of this paper is based on quantitative data from public resources and qualitative interviews. The qualitative data has been used as inspiration for topics to investigate and as argumentation for choices throughout the thesis. The acknowledged housing bubble from the mid- ’00s has been examined to benchmark with the current situation. Based on the latest price increase in all regions, it is assessed that the 

# Using Ollama

In [12]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [13]:
llm = Ollama(model="llama3", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [14]:
template = """Brug de forskellige tekst dokumenter til at svare på spørgsmålene. 
Hvis du ikke kan svare, bare sig du ikke ved det lad hver med at prøve at finde på et svar. 
Brug maks fem sætninger. Gør svaret så kort som muligt. 

{context}

Question: {question}

Helpful Answer:
"""

In [15]:
prompt = PromptTemplate.from_template(template)
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [17]:
result = chain({"query": question1})
result["result"]

Based on the plots (A-E) provided, it seems that there is a trend in Denmark where housing prices are higher in areas with fewer buyers under 30 years old. This is particularly evident for the non-capital regions (Figure 11).

Furthermore, the plots suggest that:

* There is a positive correlation between the percentage of single-buyers and housing prices (Plot B).
* For the non-capital regions, there appears to be a trend where the percentage of buyers in employment increases with higher housing prices (Plot C).
* The percentage of buyers with a long tertiary education tends to increase with higher housing prices (Plot D).
* There is also a positive correlation between median household income and housing prices (Plot E).

Overall, it seems that there are some correlations and trends observed in the data, particularly for non-capital regions. However, it's important to note that the correlation coefficients are relatively low, indicating that other factors may also be at play.

Transla

'Based on the plots (A-E) provided, it seems that there is a trend in Denmark where housing prices are higher in areas with fewer buyers under 30 years old. This is particularly evident for the non-capital regions (Figure 11).\n\nFurthermore, the plots suggest that:\n\n* There is a positive correlation between the percentage of single-buyers and housing prices (Plot B).\n* For the non-capital regions, there appears to be a trend where the percentage of buyers in employment increases with higher housing prices (Plot C).\n* The percentage of buyers with a long tertiary education tends to increase with higher housing prices (Plot D).\n* There is also a positive correlation between median household income and housing prices (Plot E).\n\nOverall, it seems that there are some correlations and trends observed in the data, particularly for non-capital regions. However, it\'s important to note that the correlation coefficients are relatively low, indicating that other factors may also be at pla

In [18]:
result = chain({"query": question2})
result["result"]

A question about Danish real estate!

According to the text, there isn't a direct answer to this question. The text only discusses seasonal adjustment methods and their application to Denmark's housing market statistics.

However, if you're interested in learning more about where house prices are rising the most in Denmark, I can suggest some online resources that might be helpful:

1. The Danish Agency for Institutions and Electronic Government (e.g., Statistikbanken.dk) publishes data on house prices and other real estate metrics.
2. Local news outlets like Berlingske or Jyllands-Posten often report on the housing market in specific regions.
3. Property websites like BoligPortal or EjerLejligheder might provide insights into local housing markets.

Keep in mind that these resources are in Danish, and you may need to use a translator to access them.

Feel free to ask if you'd like more information!

"A question about Danish real estate!\n\nAccording to the text, there isn't a direct answer to this question. The text only discusses seasonal adjustment methods and their application to Denmark's housing market statistics.\n\nHowever, if you're interested in learning more about where house prices are rising the most in Denmark, I can suggest some online resources that might be helpful:\n\n1. The Danish Agency for Institutions and Electronic Government (e.g., Statistikbanken.dk) publishes data on house prices and other real estate metrics.\n2. Local news outlets like Berlingske or Jyllands-Posten often report on the housing market in specific regions.\n3. Property websites like BoligPortal or EjerLejligheder might provide insights into local housing markets.\n\nKeep in mind that these resources are in Danish, and you may need to use a translator to access them.\n\nFeel free to ask if you'd like more information!"

In [19]:
question = "Hvordan varierer huspriserne på tværs af forskellige regioner i Danmark?"
result = chain({"query": question})
result["result"]

According to the analysis, first-time homebuyers tend to buy smaller homes in urban areas and larger homes in rural areas. Figure 9 shows that for first-time homebuyers, over a third of purchases were made in urban areas with homes under 75 square meters, while almost one-third of purchases were made in rural areas with homes over 150 square meters.

There is a significant difference between the two regions, particularly when it comes to larger homes. In urban areas, only 8% of homebuyers purchased homes over 150 square meters, whereas in rural areas, 20% of non-first-time buyers purchased homes over 150 square meters.

The analysis suggests that some of this variation can be attributed to differences in the availability of single-family homes and owner-occupied apartments between urban and rural areas.

'According to the analysis, first-time homebuyers tend to buy smaller homes in urban areas and larger homes in rural areas. Figure 9 shows that for first-time homebuyers, over a third of purchases were made in urban areas with homes under 75 square meters, while almost one-third of purchases were made in rural areas with homes over 150 square meters.\n\nThere is a significant difference between the two regions, particularly when it comes to larger homes. In urban areas, only 8% of homebuyers purchased homes over 150 square meters, whereas in rural areas, 20% of non-first-time buyers purchased homes over 150 square meters.\n\nThe analysis suggests that some of this variation can be attributed to differences in the availability of single-family homes and owner-occupied apartments between urban and rural areas.'

In [20]:
question = "Hvordan påvirker faktorer som renteniveau, økonomisk vækst og arbejdsløshedsrater typisk huspriserne?"
result = chain({"query": question})
result["result"]

Based on the provided text documents and appendices, here's an analysis of how factors like interest rates, economic growth, and unemployment rates typically affect housing prices:

**Interest Rates:**

* According to Finans Danmark (2017), a decrease in interest rates can lead to increased demand for housing, which may drive up prices.
* A study by Mørch & Rebensdorff (2005) found that a reduction in interest rates can stimulate consumption and boost the economy, potentially leading to higher housing prices.

**Economic Growth:**

* Finans Danmark's 2019 report on real credit history notes that economic growth can lead to increased demand for housing, which may drive up prices.
* A study by Stage (2016) suggests that economic growth can have a positive impact on the housing market, leading to higher prices.

**Unemployment Rates:**

* Finans Danmark's 2019 report states that high unemployment rates can negatively impact the economy and lead to decreased demand for housing, potentially

"Based on the provided text documents and appendices, here's an analysis of how factors like interest rates, economic growth, and unemployment rates typically affect housing prices:\n\n**Interest Rates:**\n\n* According to Finans Danmark (2017), a decrease in interest rates can lead to increased demand for housing, which may drive up prices.\n* A study by Mørch & Rebensdorff (2005) found that a reduction in interest rates can stimulate consumption and boost the economy, potentially leading to higher housing prices.\n\n**Economic Growth:**\n\n* Finans Danmark's 2019 report on real credit history notes that economic growth can lead to increased demand for housing, which may drive up prices.\n* A study by Stage (2016) suggests that economic growth can have a positive impact on the housing market, leading to higher prices.\n\n**Unemployment Rates:**\n\n* Finans Danmark's 2019 report states that high unemployment rates can negatively impact the economy and lead to decreased demand for housi

In [21]:
question = "Hvordan påvirker demografiske ændringer (f.eks. befolkningsvækst, alderssammensætning) boligmarkedet og priserne?"
result = chain({"query": question})
result["result"]

Based on the provided data and analysis, it can be concluded that demographic changes (e.g., population growth, age composition) affect the housing market and prices in various ways.

Firstly, an increasing population leads to a higher demand for housing, which can drive up prices. This is particularly evident in urban areas where the demand for housing often exceeds supply.

Secondly, changes in age composition may influence the type of housing that is in demand. For instance, an aging population may lead to increased demand for senior-friendly homes or age-restricted communities.

Thirdly, demographic changes can also impact the overall affordability of housing. As the population grows and ages, there may be a greater need for affordable housing options, such as social housing or subsidized rentals, which could put downward pressure on prices.

Finally, demographic changes can influence the type of neighborhoods that are in demand. For instance, an increasing number of families with 

'Based on the provided data and analysis, it can be concluded that demographic changes (e.g., population growth, age composition) affect the housing market and prices in various ways.\n\nFirstly, an increasing population leads to a higher demand for housing, which can drive up prices. This is particularly evident in urban areas where the demand for housing often exceeds supply.\n\nSecondly, changes in age composition may influence the type of housing that is in demand. For instance, an aging population may lead to increased demand for senior-friendly homes or age-restricted communities.\n\nThirdly, demographic changes can also impact the overall affordability of housing. As the population grows and ages, there may be a greater need for affordable housing options, such as social housing or subsidized rentals, which could put downward pressure on prices.\n\nFinally, demographic changes can influence the type of neighborhoods that are in demand. For instance, an increasing number of famil

In [22]:
question = "Har tilstedeværelsen af infrastruktur som skoler, offentlig transport eller indkøbsmuligheder en signifikant indflydelse på huspriserne?"
result = chain({"query": question})
result["result"]

Based on the provided documents and appendices, I can help you answer the question.

From the document "Møller, J. 5. maj 2021: Flere almene boliger i nye og eksisterende lokalplanområder?" (Appendix 3), it is mentioned that the presence of infrastructure such as schools and public transportation has a significant impact on house prices. The article suggests that the demand for housing increases when there are good local amenities, which in turn drives up prices.

Furthermore, the document "Vejledning om forsigtighed i kreditvurderingen ved belåning af boliger i vækstområder mv." (Appendix 8) also touches upon this topic. It emphasizes the importance of considering local infrastructure and amenities when evaluating the creditworthiness of a borrower.

In addition, the interview with Simon Juul Hviid and Jacob Rømeling from Nationalbanken (Appendices 1 and 2) confirms that the presence of good infrastructure has a positive impact on house prices. They mention that well-connected areas t

'Based on the provided documents and appendices, I can help you answer the question.\n\nFrom the document "Møller, J. 5. maj 2021: Flere almene boliger i nye og eksisterende lokalplanområder?" (Appendix 3), it is mentioned that the presence of infrastructure such as schools and public transportation has a significant impact on house prices. The article suggests that the demand for housing increases when there are good local amenities, which in turn drives up prices.\n\nFurthermore, the document "Vejledning om forsigtighed i kreditvurderingen ved belåning af boliger i vækstområder mv." (Appendix 8) also touches upon this topic. It emphasizes the importance of considering local infrastructure and amenities when evaluating the creditworthiness of a borrower.\n\nIn addition, the interview with Simon Juul Hviid and Jacob Rømeling from Nationalbanken (Appendices 1 and 2) confirms that the presence of good infrastructure has a positive impact on house prices. They mention that well-connected 

In [23]:
question = "Kan du udlede nogen prognoser for fremtidige huspriser baseret på historiske data og aktuelle trends?"
result = chain({"query": question})
result["result"]

Based on the provided historical data and current trends, it's possible to draw some conclusions about future house prices. However, please note that predicting exact numbers is challenging, and actual outcomes may differ from these predictions.

Here are a few observations and potential insights:

1. **Historical trend:** The data shows an upward trend in house prices over the years, with some fluctuations. This could indicate a steady increase in demand for housing.
2. **Seasonality:** There appears to be a seasonal pattern in the data, with prices tendentially increasing during certain periods (e.g., Q1 and Q3).
3. **Regional differences:** The EJS10 and EJS20 tables suggest that house price developments might vary across different regions (kommune). This could be due to local market conditions, demographics, or other factors.
4. **Quarterly vs. annual changes:** Comparing the quarterly data (EJS10) with the annual data (EJS20), we can see that some regions experienced more signific

"Based on the provided historical data and current trends, it's possible to draw some conclusions about future house prices. However, please note that predicting exact numbers is challenging, and actual outcomes may differ from these predictions.\n\nHere are a few observations and potential insights:\n\n1. **Historical trend:** The data shows an upward trend in house prices over the years, with some fluctuations. This could indicate a steady increase in demand for housing.\n2. **Seasonality:** There appears to be a seasonal pattern in the data, with prices tendentially increasing during certain periods (e.g., Q1 and Q3).\n3. **Regional differences:** The EJS10 and EJS20 tables suggest that house price developments might vary across different regions (kommune). This could be due to local market conditions, demographics, or other factors.\n4. **Quarterly vs. annual changes:** Comparing the quarterly data (EJS10) with the annual data (EJS20), we can see that some regions experienced more 

# Exporting the model

In [24]:
import pickle
pickle.dump(llm, open('./App/models/LLMModel.pkl', 'wb'))